In [1]:
from mongoconnection import *
import pandas as pd

In [2]:
dat = pd.read_csv('../data/cbd_info2.csv')

In [3]:
def lista ():
    a = dat.Effects.unique()
    b = [i.split(",") for i in a]
    z = []
    for j in b:
        for k in j:
            z.append(k)
            z = list(set(z))
            for x in z:
                if x == "None" or x=="Dry" or x=="Mouth":
                    z.remove(x)
    return z

In [4]:
filt = {"mood":{"$regex":"Happy"}}
project = {"mood":1, "product":1, "_id":0}
result = db.cbd_info.find(filt, project)
len(list(result))

1825

In [5]:
db.cbd_info.count({"mood": {"$regex": "Relaxed"}})

<ipython-input-5-b31db318e967>:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  db.cbd_info.count({"mood": {"$regex": "Relaxed"}})


1693

In [6]:
def shop_mood(moods):
    filt = {"mood":{"$regex":f"{moods}"}}
    project = {"mood": 1, "_id":0, "product":1}
    result = db.cbd_info.find(filt, project)
    results = list(result)
    for products in results:
            filt = {"product":{"$regex":f"{products}"}}
            project = {"mood": 1, "product":1, "shop":1, "CBD percentage":1, "_id":0}
            shops = db.cbd_shops.find(filt, project) #seguir iterando para hacer 
                                                    #query en ratings collection?
    return list(shops)

In [7]:
shop_mood("Happy")

[]

In [18]:
def check_exists(query, collection, *pro):
    res = read_coll(collection, query, pro)
    if len(res) > 0:
        return True
    else:
        return False

In [19]:
def shop_mood(moods):
    filt = {"mood":{"$regex":f"{moods}"}}
    project = {"_id":0, "product":1}
    result = db.cbd_info.find(filt, project)
    results= list(result)
    
    lista_res = []
    for i in results:
        q = {"product":i["product"]}
        pro = {"_id":0} 
        if check_exists(q, "cbd_shops"):
            lista_res.append(read_coll("cbd_shops",q,  pro))

    return lista_res

shop_mood("Creative")

[[{'product': 'Amnesia',
   'CBD percentage': '4%',
   'price': '13,50 €',
   'shop': 'https://top-garden.net/es/flor-de-cbd/flor-de-cbd-amnesia-4-cbd-flor-de-canamo'},
  {'product': 'Amnesia',
   'CBD percentage': '4%',
   'price': '11,49 €',
   'shop': 'https://top-garden.net/es/flor-de-cbd/flor-de-cbd-amnesia-4-cbd-flor-de-canamo'},
  {'product': 'Amnesia',
   'CBD percentage': '6%',
   'price': '10,99 €',
   'shop': 'https://top-garden.net/es/flor-de-cbd/flor-de-cbg-amnesia-6-cbg-flor-de-canamo'}],
 [{'product': 'Caramel Candy',
   'CBD percentage': '3.5%',
   'price': '11,99 €',
   'shop': 'https://top-garden.net/es/flor-de-cbd/flor-de-cbd-caramel-candy-35-cbd-flor-de-canamo'}],
 [{'product': 'Royal Cheese',
   'CBD percentage': '5%',
   'price': '11,99 €',
   'shop': 'https://top-garden.net/es/flor-de-cbd/flor-de-cbd-royal-cheese-5-cbd-flor-de-canamo'}],
 [{'product': 'Gelato',
   'CBD percentage': '5%',
   'price': '12,99 €',
   'shop': 'https://top-garden.net/es/flor-de-cbd/flo

In [128]:
""" STACKOVERFLOW:
def findmood():
    
db.users.aggregate([{$lookup:
  {
        from: "contacts",
        localField: "number",
        foreignField: "phonenumber",
        as: "same"
    }
   },
   {
      $match: { "same": { $ne: [] } }
   }
])

"""

' STACKOVERFLOW:\ndef findmood():\n    \ndb.users.aggregate([{$lookup:\n  {\n        from: "contacts",\n        localField: "number",\n        foreignField: "phonenumber",\n        as: "same"\n    }\n   },\n   {\n      $match: { "same": { $ne: [] } }\n   }\n])\n\n'